# **Top IMDb Movies Analysis and Prediction**


**PYTHON FOR DATA ANALYTICS [CS6102]**

By
*   Gurucharan S (23MSP3074)
*   Dharshani A (23MSP3068)










**AIM:**
To analyze and visualize the data from the IMDB Top 250 movies, establish database connectivity, and implement linear regression for predicting movie ratings.

**DESCRIPTION:**
The main goal of this project is to gain insights from the IMDb top 250 movies,preprocess the data for analysis, visualize the data patterns, establish a connection with an SQLite database to store and retrieve movie data, and implement a linear regression model to predict movie ratings.


**Source of the dataset:** IMDB (as inferred from the filename "imdbTop250.csv").



**Dataset description:** The dataset contains various columns such as title, rating, score, and runtime of the top 250 movies on IMDB.

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import shutil

from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
# from google.colab import drive


In [5]:
# Mount Google Drive

src = "C:\Users\dhars\Desktop\TRIMESTER 1\Trimester 1 Projects\PYTHON USE CASE\imdbTop250.csv"
dst = "C:\Users\dhars\Desktop\TRIMESTER 1\Trimester 1 Projects\PYTHON USE CASE\imdbTop250m.csv"

# Copy file from source location to destination location
shutil.copy(src, dst)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (7373071.py, line 3)

**Data Pre-Processing:** The data preprocessing involves reading the dataset, slicing it to get the first 50 movies, and handling missing values by dropping rows that have any.

**Exploratory Data Analysis(Visualization):**
The exploratory data analysis comprises several visual representations:

*   Boxplot to identify outliers in votes.
*   Scatter plot for comparing votes against runtime.
*   Bar plot to show top 20 movie ratings.
*   Histogram for comparing movie rankings with their runtime.



**Storing Data in Database:** The processed data is stored in an SQLite database named "usecase.db". The movie details are saved under the table "TopIMDBMovies". The database further allows for querying and extraction of movies based on different criteria related to their ratings.
An SQLite database named "usecase.db" is created.
The data from the dataset (columns 1 to 10) is stored in a table named "TopIMDBMovies" in the SQLite database.
Various SQL queries are executed to retrieve and analyze data. For example:

*   Movies with a rating of less than 8.
*   The count of movies with a rating greater than 8.
*   The movie with the highest rating.
*   The movie with the lowest rating.

**Building Predictive model and testing:** Linear regression is a statistical method that models the relationship between a dependent variable and one or more independent variables. In predictive analysis, it is used to forecast values based on known data, allowing for the prediction of outcomes based on input variables.

In the below code linear regression is being applied to predict movie "Rating" based on its "Score". By splitting the data into training and testing sets, the model is trained on a subset of the data and then evaluated on unseen data. This approach, commonly used in predictive analysis, helps in understanding how well the model will generalize to new, previously unseen data.

In [ ]:
# Define a class for the use case
class Usecase:
    @staticmethod
    def dataPreprocessing():
        # Read the CSV file into a DataFrame
        df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/imdbTop250m.csv')
        df = df.iloc[:51]  # Filter the first 50 rows
        df = df.dropna()  # Drop rows with missing values
        return df

    df1 = dataPreprocessing()


# Data Visualization code snippets

    @staticmethod
    def visualization_outliers(df):
        # Visualize outliers using a boxplot
        plt.figure(figsize=(10, 6))
        plt.xlabel("Total votes")
        plt.title("Identifying Outliers")
        sns.boxplot(y="Votes", data=df)
        plt.show()

    @staticmethod
    def visualization_scatterplot(df):
        # Visualize data using a scatterplot
        plt.figure(figsize=(10, 6))
        plt.xlabel("Total Votes")
        plt.ylabel("Runtime")
        plt.title("Comparison of Total votes and Runtime")
        sns.scatterplot(x="Votes", y="RunTime", data=df)
        plt.show()

    @staticmethod
    def visualization_barplot(df):
        # Visualize top 10 movies and their ratings using a barplot
        df_subset = df.iloc[0:10]
        plt.figure(figsize=(10, 6))
        plt.title("Movies and their Ratings")
        sns.barplot(x="Title", y="Rating", data=df_subset)
        plt.xticks(rotation=90)  # Rotate x-axis labels for clarity
        plt.show()

    @staticmethod
    def visualization_histogram(df):
        # Visualize data using a histogram
        plt.figure(figsize=(10, 6))
        plt.xlabel("Ranking")
        plt.ylabel("Runtime")
        plt.title("Comparison of Total votes and Runtime")
        sns.histplot(data=df, x="Ranking", y="RunTime")
        plt.show()


# Database Connectivity code snippets

    @staticmethod
    def database_connectivity(df1):
        # Extract columns 1 to 10 from the df1 DataFrame
        df2 = df1.iloc[:, 1:11]

        # Create a connection to the SQLite database
        conn = sqlite3.connect('usecase.db')
        cursor = conn.cursor()

        # Write the data to the database
        df2.to_sql(name="TopIMDBMovies", con=conn, if_exists='replace', index=False)

        # Query to retrieve records where RATING is less than 8
        select_query = "SELECT * FROM TopIMDBMovies WHERE RATING < 8"

        # Query to count the number of movies with a RATING greater than 8
        count_query = "SELECT COUNT(TITLE) AS BEST_MOVIES FROM TopIMDBMovies WHERE RATING > 8"

        # Query to display the movie with the highest rating
        highest_query = "SELECT * FROM TopIMDBMovies WHERE Rating IN (SELECT MAX(Rating) FROM TopIMDBMovies )"

        # Query to display the movie with the minimum rating
        lowest_query = "SELECT * FROM TopIMDBMovies WHERE Rating IN (SELECT MIN(Rating) FROM TopIMDBMovies )"

        # Execute the queries and fetch results
        cursor.execute(select_query)
        results_low_rating = cursor.fetchall()

        cursor.execute(count_query)
        result_best_movies = cursor.fetchall()

        cursor.execute(highest_query)
        result_highest_rating = cursor.fetchall()

        cursor.execute(lowest_query)
        result_lowest_rating = cursor.fetchall()

        # Get column names (headers) for the first query
        headers_low_rating = [description[0] for description in conn.execute(select_query).description]

        # Get column names (headers) for the second query
        headers_best_movies = [description[0] for description in conn.execute(count_query).description]

        # Get column names (headers) for the third query
        headers_highest_rating = [description[0] for description in conn.execute(highest_query).description]

        # Get column names (headers) for the fourth query
        headers_lowest_rating = [description[0] for description in conn.execute(lowest_query).description]

        # Print results in tabular form using tabulate
        print("Movies with Rating < 8:")
        print(tabulate(results_low_rating, headers_low_rating, tablefmt="grid"))
        print("\n")

        print("Number of Best Movies (Rating > 8):")
        print(tabulate(result_best_movies, headers_best_movies, tablefmt="grid"))
        print("\n")

        print("Movie with the Highest Rating:")
        print(tabulate(result_highest_rating, headers_highest_rating, tablefmt="grid"))
        print("\n")

        print("Movie with the Lowest Rating:")
        print(tabulate(result_lowest_rating, headers_lowest_rating, tablefmt="grid"))

        # Commit and close the connection
        conn.commit()
        cursor.close()
        conn.close()


# Data preparation for linear regression and rating prediction code snippet

    @staticmethod
    def dataset_linearregression(df):
      # Prepare data for linear regression
        y = df["Rating"]
        x = df[["Score"]]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

# Create an instance of the Usecase class
usecase_instance = Usecase()

# Perform data preprocessing
df1 = usecase_instance.dataPreprocessing()




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
usecase_instance.df1

In [ ]:
usecase_instance.visualization_barplot(df1)

In [ ]:
usecase_instance.visualization_scatterplot(df1)

In [ ]:
usecase_instance.visualization_outliers(df1)

In [ ]:
usecase_instance.visualization_histogram(df1)

In [ ]:
# Call database connectivity function
usecase_instance.database_connectivity(df1)

In [ ]:
#Setting value for x and y
df2 = usecase_instance.df1
y= df2["Rating"]
x= df2[["Score"]]
x_train,x_test, y_train,y_test = train_test_split(x,y, test_size=0.3, random_state = 10)
x_train

In [ ]:
slr= LinearRegression()
slr.fit(x_train,y_train)

In [ ]:
#Print model coefficients
print('Intercept:',slr.intercept_)
print('Coefficient:', slr.coef_)

In [ ]:
'''A linear regression model is implemented to predict movie ratings based on their scores.
The dataset is split into training and testing sets using a 70:30 split ratio.'''


In [ ]:
#LINE OF Best Fit

# Assuming x_train and y_train are your data
x_train = np.array(x_train).reshape(-1, 1)  # Reshaping if x_train is a 1D array

# Create a linear regression model
model = LinearRegression()

# Fit the model to the data
model.fit(x_train, y_train)

# Predict y values for the x_train data
y_pred = model.predict(x_train)

# Plot the original data points
plt.scatter(x_train, y_train, color='blue')

# Plot the line of best fit
plt.plot(x_train, y_pred, color='green')

plt.show()

In [ ]:
y_pred_slr = slr.predict(x_test)
print('Prediction result:{}'.format(y_pred_slr))

In [ ]:
#actual value and predicted value
slr_diff = pd.DataFrame({'Actual value': y_test,'Predicted Value': y_pred_slr})
slr_diff

In [ ]:
#predict for any value
slr.predict([[13]])

In [ ]:
#R squared value
from sklearn.metrics import accuracy_score
print('R squared value: {:2f}'.format(slr.score(x,y)*100))

In [ ]:
# Calculate the Mean Absolute Error (MAE) between the actual and predicted values
MeanAbsErr = metrics.mean_absolute_error(y_test, y_pred_slr)

# Calculate the Mean Squared Error (MSE) between the actual and predicted values
MeanSquErr = metrics.mean_squared_error(y_test, y_pred_slr)

# Calculate the Root Mean Squared Error (RMSE) between the actual and predicted values
RootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_slr))

# Display the calculated error metrics with precision up to three decimal places
print('Absolute Mean error:', round(MeanAbsErr, 3))
print('Mean Square error:', round(MeanSquErr, 3))
print('Root Mean Square error:', round(RootMeanSqErr, 3))

**CONCLUSION:**
The data from the IMDB Top 250 movies was successfully analyzed and visualized. Database operations were performed to store and query the data. A linear regression model was initiated for predicting movie ratings based on their scores.

Overall, the project gives a comprehensive overview of how to handle, process, and analyze data from a real-world dataset. The combination of data visualization, database operations, and the beginning of predictive modeling offers a solid foundation for further enhancements and deeper analyses.